# Homework Classification

In [1014]:
import pandas as pd
import numpy as np
import random

In [1015]:
#!wget https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-02-car-price/data.csv

In [1016]:
df = pd.read_csv('data.csv')

In [1017]:
df.head().T

,0,1,2,3,4
Make,BMW,BMW,BMW,BMW,BMW
Model,1 Series M,1 Series,1 Series,1 Series,1 Series
Year,2011,2011,2011,2011,2011
Engine Fuel Type,premium unleaded (required),premium unleaded (required),premium unleaded (required),premium unleaded (required),premium unleaded (required)
Engine HP,335.0,300.0,300.0,230.0,230.0
Engine Cylinders,6.0,6.0,6.0,6.0,6.0
Transmission Type,MANUAL,MANUAL,MANUAL,MANUAL,MANUAL
Driven_Wheels,rear wheel drive,rear wheel drive,rear wheel drive,rear wheel drive,rear wheel drive
Number of Doors,2.0,2.0,2.0,2.0,2.0
Market Category,"Factory Tuner,Luxury,High-Performance","Luxury,Performance","Luxury,High-Performance","Luxury,Performance",Luxury


In [1018]:
#Select only the features from above (github)
df = df[['Make','Model','Year', 'Engine HP', 'Engine Cylinders', 'Transmission Type', 'Vehicle Style', 'highway MPG', 'city mpg', 'MSRP']]

In [1019]:
#transform their names using the next line (github)
df.columns = [col.lower().replace(' ','_') for col in df.columns]

In [1020]:
df.isna().sum()

make                  0
model                 0
year                  0
engine_hp            69
engine_cylinders     30
transmission_type     0
vehicle_style         0
highway_mpg           0
city_mpg              0
msrp                  0
dtype: int64

In [1021]:
#Fill in the missing values of the selected features with 0
df = df.fillna(0)

In [1022]:
#Rename MSRP variable to price, "columns=" is necessary!
df.rename(columns = {'msrp':'price'}, inplace = True)

In [1023]:
df.columns

Index(['make', 'model', 'year', 'engine_hp', 'engine_cylinders',
       'transmission_type', 'vehicle_style', 'highway_mpg', 'city_mpg',
       'price'],
      dtype='object')

### Question 1
What is the most frequent observation (mode) for the column transmission_type?

In [1024]:
df['transmission_type'].value_counts()

transmission_type
AUTOMATIC           8266
MANUAL              2935
AUTOMATED_MANUAL     626
DIRECT_DRIVE          68
UNKNOWN               19
Name: count, dtype: int64

### Question 2
Features with biggest correlation


In [1025]:
#Create the correlation matrix
df.head()

,make,model,year,engine_hp,engine_cylinders,transmission_type,vehicle_style,highway_mpg,city_mpg,price
0,BMW,1 Series M,2011,335.0,6.0,MANUAL,Coupe,26,19,46135
1,BMW,1 Series,2011,300.0,6.0,MANUAL,Convertible,28,19,40650
2,BMW,1 Series,2011,300.0,6.0,MANUAL,Coupe,28,20,36350
3,BMW,1 Series,2011,230.0,6.0,MANUAL,Coupe,28,18,29450
4,BMW,1 Series,2011,230.0,6.0,MANUAL,Convertible,28,18,34500


In [1026]:
#df[['year','engine_cylinders']].corrwith(df.engine_hp)
#df[['engine_cylinders', 'city_mpg']].corrwith(df.highway_mpg)

In [1027]:
#In order to create correlation matrix, need to be numerical
numerical = list(df.dtypes[(df.dtypes != 'object')].index)
numerical

['year', 'engine_hp', 'engine_cylinders', 'highway_mpg', 'city_mpg', 'price']

In [1028]:
categorical = list(df.dtypes[df.dtypes == 'object'].index)
categorical

['make', 'model', 'transmission_type', 'vehicle_style']

In [1029]:
#Creating correlation matrix
df[numerical].corr()

,year,engine_hp,engine_cylinders,highway_mpg,city_mpg,price
year,1.000000,0.338714,-0.040708,0.258240,0.198171,0.227590
engine_hp,0.338714,1.000000,0.774851,-0.415707,-0.424918,0.650095
engine_cylinders,-0.040708,0.774851,1.000000,-0.614541,-0.587306,0.526274
highway_mpg,0.258240,-0.415707,-0.614541,1.000000,0.886829,-0.160043
city_mpg,0.198171,-0.424918,-0.587306,0.886829,1.000000,-0.157676
price,0.227590,0.650095,0.526274,-0.160043,-0.157676,1.000000


In [1030]:
#Get the average price
avg_price = df['price'].mean()
avg_price

40594.737032063116

In [1031]:
#Create  binary column with above and below average price
df['above_average'] = (df['price'] > avg_price).astype(int)

In [1032]:
df[numerical].corrwith(df['above_average'])

year                0.318311
engine_hp           0.660518
engine_cylinders    0.448907
highway_mpg        -0.133440
city_mpg           -0.149233
price               0.481489
dtype: float64

In [1033]:
#Split the data  in train/val/test sets with 60%/20%/20%
from sklearn.model_selection import train_test_split


X_full_train, X_test = train_test_split(df, test_size = 0.2, train_size = 0.8,  random_state=42)

X_train, X_val = train_test_split(X_full_train, test_size = 0.25, train_size = 0.75, random_state=42)


In [1034]:
X_full_train.reset_index(drop=True,inplace=True)
X_test.reset_index(drop=True,inplace=True)
X_train.reset_index(drop=True,inplace=True)
X_val.reset_index(drop=True,inplace=True)

In [1035]:
#Create target series
Y_full_train = X_full_train['above_average']
Y_test = X_test['above_average']
Y_train = X_train['above_average']
Y_val = X_val['above_average']

In [1036]:
X_full_train.columns

Index(['make', 'model', 'year', 'engine_hp', 'engine_cylinders',
       'transmission_type', 'vehicle_style', 'highway_mpg', 'city_mpg',
       'price', 'above_average'],
      dtype='object')

In [1037]:
del X_full_train['above_average']
del X_test['above_average']
del X_train['above_average']
del X_val['above_average']

del X_full_train['price']
del X_test['price']
del X_train['price']
del X_val['price']


### Question 3
Feature with the lowest mutual information score


In [1038]:
#https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mutual_info_score.html
from sklearn.metrics import mutual_info_score

In [1039]:
categorical

['make', 'model', 'transmission_type', 'vehicle_style']

In [1040]:
def mutual_info_score_cars(column):
    return round(mutual_info_score(column, Y_train),2)

In [1041]:
#Apply goes throuch each column and applies the function
X_train[categorical].apply(mutual_info_score_cars)

make                 0.24
model                0.46
transmission_type    0.02
vehicle_style        0.08
dtype: float64

### Question 4
* Now let's train a logistic regression.
* Remember that we have several categorical variables in the dataset. Include them using one-hot encoding.
* Fit the model on the training dataset.
    * To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
    * model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)
* Calculate the accuracy on the validation dataset and round it to 2 decimal digits.

In [1042]:
#https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.DictVectorizer.html
from sklearn.feature_extraction import DictVectorizer
#https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
from sklearn.linear_model import LogisticRegression

In [1043]:
#Create instance of DictVectorizer
vectorizer = DictVectorizer(sparse=False)
#Transform dataframe to list of dictionaries, vectorizer needs that format, orient needs to be records
X_train_dicts = X_train.to_dict(orient='records')
#fit = recognize features , transform = create multidimensional array
X_train_vectorized = vectorizer.fit_transform(X_train_dicts)


In [1044]:
#Create instance of model
model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)
model.fit(X_train_vectorized, Y_train)

LogisticRegression(C=10, max_iter=1000, random_state=42, solver='liblinear')

In [1045]:
#Use vectorizer to transform X_val
X_val_dicts = X_val.to_dict(orient = 'records')
X_val_vectorized = vectorizer.transform(X_val_dicts)
val_predict = model.predict(X_val_vectorized)
val_predict_proba = model.predict_proba(X_val_vectorized)

In [1046]:
df_val_result = pd.DataFrame()
df_val_result['y_val'] = Y_val
df_val_result['y_val_predicted'] = val_predict
df_val_result['y_val_predicted_proba'] = val_predict_proba[:,1]



In [1047]:
df_val_result['match'] = (df_val_result.y_val == df_val_result.y_val_predicted)

In [1048]:
accuracy_val_all_features = round(df_val_result['match'].mean(),2)
accuracy_val_all_features

0.93

In [1049]:
#The accuracy is almost 100%?? --> Checking with test data
X_test_dicts = X_test.to_dict(orient = 'records')
X_test_vectorized = vectorizer.transform(X_test_dicts)
test_predict = model.predict(X_test_vectorized)
test_predict_proba = model.predict_proba(X_val_vectorized)


In [1050]:
df_test_result = pd.DataFrame()
df_test_result['y_test'] = Y_test
df_test_result['y_test_predicted'] = test_predict
df_test_result['y_test_predicted_proba'] = test_predict_proba[:,1]


In [1051]:
df_test_result['match'] = (df_test_result.y_test == df_test_result.y_test_predicted)

In [1052]:
df_test_result['match'].mean()

0.9328577423415862

### Question 5
* Let's find the least useful feature using the feature elimination technique.
* Train a model with all these features (using the same parameters as in Q4).
* Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
* For each feature, calculate the difference between the original accuracy and the accuracy without the feature.

Remove year

In [1053]:
X_train__year  = X_train.drop(columns={'year'})
X_val__year  = X_val.drop(columns={'year'})
X_test__year  = X_test.drop(columns={'year'})

print(list(X_train__year.columns))
print(list(X_val__year.columns))
print(list(X_test__year.columns))


['make', 'model', 'engine_hp', 'engine_cylinders', 'transmission_type', 'vehicle_style', 'highway_mpg', 'city_mpg']
['make', 'model', 'engine_hp', 'engine_cylinders', 'transmission_type', 'vehicle_style', 'highway_mpg', 'city_mpg']
['make', 'model', 'engine_hp', 'engine_cylinders', 'transmission_type', 'vehicle_style', 'highway_mpg', 'city_mpg']


In [1054]:
#One hot encoding

vectorizer_no_year = DictVectorizer(sparse=False)
X_train__year_dicts = X_train__year.to_dict(orient = 'records')
vectorized_no_year_train = vectorizer_no_year.fit_transform(X_train__year_dicts)
model_no_year = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)
model_no_year.fit(vectorized_no_year_train, Y_train)

X_val__year_dicts = X_val__year.to_dict(orient = 'records')
vectorized_no_year_val = vectorizer_no_year.transform(X_val__year_dicts)
Y_val_no_year_pred = model_no_year.predict(vectorized_no_year_val)


df_val_result_no_year = pd.DataFrame()
df_val_result_no_year['y_val'] = Y_val
df_val_result_no_year['y_val_predicted'] = Y_val_no_year_pred
df_val_result_no_year['match'] = (df_val_result_no_year['y_val'] == df_val_result_no_year['y_val_predicted'])

#no_year_accuracy 
no_year_accuracy = df_val_result_no_year['match'].mean()
diff_accuracy_noyear = accuracy_val_all_features - no_year_accuracy
print(diff_accuracy_noyear)

diff_accuracy_noyear_abs = abs(diff_accuracy_noyear)
diff_accuracy_noyear_abs

-0.01670583298363404


0.01670583298363404

Remove engine_hp

In [1055]:
X_train__engine_hp  = X_train.drop(columns={'engine_hp'})
X_val__engine_hp  = X_val.drop(columns={'engine_hp'})
X_test__engine_hp  = X_test.drop(columns={'engine_hp'})

print(list(X_train__engine_hp.columns))
print(list(X_val__engine_hp.columns))
print(list(X_test__engine_hp.columns))


['make', 'model', 'year', 'engine_cylinders', 'transmission_type', 'vehicle_style', 'highway_mpg', 'city_mpg']
['make', 'model', 'year', 'engine_cylinders', 'transmission_type', 'vehicle_style', 'highway_mpg', 'city_mpg']
['make', 'model', 'year', 'engine_cylinders', 'transmission_type', 'vehicle_style', 'highway_mpg', 'city_mpg']


In [1056]:
#One hot encoding

vectorizer_no_year = DictVectorizer(sparse=False)
X_train__year_dicts = X_train__engine_hp.to_dict(orient = 'records')
vectorized_no_year_train = vectorizer_no_year.fit_transform(X_train__year_dicts)
model_no_year = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)
model_no_year.fit(vectorized_no_year_train, Y_train)

X_val__year_dicts = X_val__engine_hp.to_dict(orient = 'records')
vectorized_no_year_val = vectorizer_no_year.transform(X_val__year_dicts)
Y_val_no_year_pred = model_no_year.predict(vectorized_no_year_val)


df_val_result_no_year = pd.DataFrame()
df_val_result_no_year['y_val'] = Y_val
df_val_result_no_year['y_val_predicted'] = Y_val_no_year_pred
df_val_result_no_year['match'] = (df_val_result_no_year['y_val'] == df_val_result_no_year['y_val_predicted'])

#no_year_accuracy 
no_year_accuracy = df_val_result_no_year['match'].mean()
diff_accuracy_noyear = accuracy_val_all_features - no_year_accuracy
print(diff_accuracy_noyear)

0.006374318086445729


Remove transmission_type

In [1057]:
#no_transmission_type_accuracy
X_train__transmission_type  = X_train.drop(columns={'transmission_type'})
X_val__transmission_type  = X_val.drop(columns={'transmission_type'})
X_test__transmission_type  = X_test.drop(columns={'transmission_type'})

print(list(X_train__transmission_type.columns))
print(list(X_val__transmission_type.columns))
print(list(X_test__transmission_type.columns))

['make', 'model', 'year', 'engine_hp', 'engine_cylinders', 'vehicle_style', 'highway_mpg', 'city_mpg']
['make', 'model', 'year', 'engine_hp', 'engine_cylinders', 'vehicle_style', 'highway_mpg', 'city_mpg']
['make', 'model', 'year', 'engine_hp', 'engine_cylinders', 'vehicle_style', 'highway_mpg', 'city_mpg']


In [1058]:
#One hot encoding

vectorizer_no_year = DictVectorizer(sparse=False)
X_train__year_dicts = X_train__transmission_type.to_dict(orient = 'records')
vectorized_no_year_train = vectorizer_no_year.fit_transform(X_train__year_dicts)
model_no_year = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)
model_no_year.fit(vectorized_no_year_train, Y_train)

X_val__year_dicts = X_val__transmission_type.to_dict(orient = 'records')
vectorized_no_year_val = vectorizer_no_year.transform(X_val__year_dicts)
Y_val_no_year_pred = model_no_year.predict(vectorized_no_year_val)


df_val_result_no_year = pd.DataFrame()
df_val_result_no_year['y_val'] = Y_val
df_val_result_no_year['y_val_predicted'] = Y_val_no_year_pred
df_val_result_no_year['match'] = (df_val_result_no_year['y_val'] == df_val_result_no_year['y_val_predicted'])

#no_year_accuracy 
no_year_accuracy = df_val_result_no_year['match'].mean()
diff_accuracy_noyear = accuracy_val_all_features - no_year_accuracy
print(diff_accuracy_noyear)

-0.015027276542173729


Remove city_mpg

In [1059]:

X_train__city_mpg  = X_train.drop(columns={'city_mpg'})
X_val__city_mpg  = X_val.drop(columns={'city_mpg'})
X_test__city_mpg  = X_test.drop(columns={'city_mpg'})

print(list(X_train__city_mpg.columns))
print(list(X_val__city_mpg.columns))
print(list(X_test__city_mpg.columns))

['make', 'model', 'year', 'engine_hp', 'engine_cylinders', 'transmission_type', 'vehicle_style', 'highway_mpg']
['make', 'model', 'year', 'engine_hp', 'engine_cylinders', 'transmission_type', 'vehicle_style', 'highway_mpg']
['make', 'model', 'year', 'engine_hp', 'engine_cylinders', 'transmission_type', 'vehicle_style', 'highway_mpg']


In [1060]:
#One hot encoding

vectorizer_no_year = DictVectorizer(sparse=False)
X_train__year_dicts = X_train__city_mpg.to_dict(orient = 'records')
vectorized_no_year_train = vectorizer_no_year.fit_transform(X_train__year_dicts)
model_no_year = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)
model_no_year.fit(vectorized_no_year_train, Y_train)

X_val__year_dicts = X_val__city_mpg.to_dict(orient = 'records')
vectorized_no_year_val = vectorizer_no_year.transform(X_val__year_dicts)
Y_val_no_year_pred = model_no_year.predict(vectorized_no_year_val)


df_val_result_no_year = pd.DataFrame()
df_val_result_no_year['y_val'] = Y_val
df_val_result_no_year['y_val_predicted'] = Y_val_no_year_pred
df_val_result_no_year['match'] = (df_val_result_no_year['y_val'] == df_val_result_no_year['y_val_predicted'])

#no_year_accuracy 
no_year_accuracy = df_val_result_no_year['match'].mean()
diff_accuracy_noyear = accuracy_val_all_features - no_year_accuracy
print(diff_accuracy_noyear)

-0.016286193873268906


### Question 6

* For this question, we'll see how to use a linear regression model from Scikit-Learn.
* We'll need to use the original column price. Apply the logarithmic transformation to this column.
* Fit the Ridge regression model on the training data with a solver 'sag'. Set the seed to 42.
* This model also has a parameter alpha. Let's try the following values: [0, 0.01, 0.1, 1, 10].
* Round your RMSE scores to 3 decimal digits.


In [1061]:
#1. Create train,val, test
    # 1.1 Create Y_train, Y_val, Y_test, LOGARITHMIC TRANSFORMATION
    # 1.2 Delete them from their corresponding datasets
#2. Create one hot encoding
    #2.1 create dict vectorizer instance
    #2.2 transform X_train into dictionary
    #2.2 fit dict vectorizer with X_train_dict
    #2.3 transform X_train_dict
#3. Create model instance
    #3.1 fit model to X_train_dict
    #3.2 transform X_val to dictionary
    #3.3 fit X_val to DictVectorizer
    #3.4 precdic X_val_dict using the linear regression model
#4. Fit model

In [1062]:
df = df.drop(columns={'above_average'})

In [1063]:
#Train , val, est split (0.6,0.2,0.2)

X_full_train, X_test = train_test_split(df, train_size = 0.8, test_size = 0.2)
X_train, X_val = train_test_split(X_full_train, train_size= 0.75, test_size = 0.25)

X_train.reset_index(drop = True, inplace= True)
X_val.reset_index(drop = True, inplace= True)
X_test.reset_index(drop = True, inplace= True)

In [1064]:
#Create Target variable
Y_train = X_train['price']
Y_test = X_test['price']
Y_val = X_val['price']

In [1065]:
#Make target variable logarithimic (this brings it more to the normal distribution with wich the models work better)
Y_train_log = np.log1p(Y_train)
Y_test_log = np.log1p(Y_test)
Y_val_log = np.log1p(Y_val)

In [1066]:
#Remove the target variable from X_train, X_val and X_test
del X_train['price']
del X_val['price']
del X_test['price']

In [1070]:
#Create one hot encoding
vectorizer = DictVectorizer()
X_train_dic = X_train.to_dict(orient = 'records')
X_train_vec = vectorizer.fit_transform(X_train_dic)

In [1071]:
#Create the ridge regression model
from sklearn.linear_model import Ridge
ridge_model_0 = Ridge(alpha = 0, solver='sag', random_state=42)
ridge_model_0_01 = Ridge(alpha = 0.01, solver='sag', random_state=42)
ridge_model_0_1 = Ridge(alpha = 0.1, solver='sag', random_state=42)
ridge_model_1 = Ridge(alpha = 1, solver='sag', random_state=42)
ridge_model_10 = Ridge(alpha = 10, solver='sag', random_state=42)

In [1073]:
#Fit the models
ridge_model_0.fit(X_train_vec, Y_train_log)
ridge_model_0_01.fit(X_train_vec, Y_train_log)
ridge_model_0_1.fit(X_train_vec, Y_train_log)
ridge_model_1.fit(X_train_vec, Y_train_log)
ridge_model_10.fit(X_train_vec, Y_train_log)

Ridge(alpha=10, random_state=42, solver='sag')

In [1076]:
#Create vectorized validation
X_val_dic = X_val.to_dict(orient = 'records')
vectorized_val = vectorizer.transform(X_val_dic)

In [1077]:
#Create predictions
Y_pred_0 = ridge_model_0.predict(vectorized_val)
Y_pred_0_01 = ridge_model_0_01.predict(vectorized_val)
Y_pred_0_1 = ridge_model_0_1.predict(vectorized_val)
Y_pred_1 = ridge_model_1.predict(vectorized_val)
Y_pred_10 = ridge_model_10.predict(vectorized_val)


In [1088]:
#Calculate the root mean squared error for each model
#https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html

from sklearn.metrics import mean_squared_error

rmse_0 = mean_squared_error(Y_val_log,Y_pred_0, squared= False)
rmse_0_01 = mean_squared_error(Y_val_log,Y_pred_0_01, squared= False)
rmse_0_1 = mean_squared_error(Y_val_log,Y_pred_0_1, squared= False)
rmse_1 = mean_squared_error(Y_val_log,Y_pred_1, squared= False)
rmse_10 = mean_squared_error(Y_val_log,Y_pred_10, squared= False)

print('rmse_0',rmse_0)
print('rmse_0_01', rmse_0_01)
print('rmse_0_1', rmse_0_1)
print('rmse_1', rmse_1)
print('rmse_10', rmse_10)

print(rmse_0 > rmse_0_01)


rmse_0 0.26386707178837243
rmse_0_01 0.2638385341309458
rmse_0_1 0.263615785362297
rmse_1 0.27086955726207357
rmse_10 0.3338838328961651
True
